In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install tritonclient\[all\]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.8/247.8 kB 5.5 MB/s eta 0:00:00


In [3]:
import numpy as np
import tritonclient.http as httpclient
from PIL import Image
from tritonclient.utils import triton_to_np_dtype

In [24]:
from PIL import Image
from torchvision import transforms

def mobi_preprocess(img_path="img1.jpg"):
    # Open the image using PIL
    img = Image.open(img_path)

    # Define the preprocessing steps
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to 224x224
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(
            mean=[0.46295794, 0.46194877, 0.4847407],
            std=[0.19444681, 0.19439201, 0.19383532]
        ),  # Normalize
    ])

    # Apply the preprocessing steps to the image and convert to numpy array
    img_tensor = preprocess(img)
    img_numpy = img_tensor.numpy()

    return img_numpy


In [25]:
def softmax(logits):
    exp_logits = np.exp(logits - np.max(logits))  # Subtract max for numerical stability
    return exp_logits / np.sum(exp_logits)

def process_predictions(predictions):
    results = []
    for pred in predictions:
        score, class_idx, label = pred.decode().split(':')
        results.append((float(score), int(class_idx), label))

    # Convert list of scores to an array for softmax calculation
    scores = np.array([result[0] for result in results])
    probabilities = softmax(scores)

    # Associate probabilities with corresponding class labels and indices
    for i, result in enumerate(results):
        results[i] = (probabilities[i], result[1], result[2])

    # Sort by probabilities in descending order
    results.sort(key=lambda x: x[0], reverse=True)

    return results



Inference

In [26]:
transformed_img = mobi_preprocess("/content/drive/MyDrive/ML-DL/MLOps-Production/test_2294_10/car/car_0000.jpg")

# Setting up client
client = httpclient.InferenceServerClient(url="35.240.142.147:8000")

inputs = httpclient.InferInput("input", transformed_img.shape, datatype="FP32")
inputs.set_data_from_numpy(transformed_img)

outputs = httpclient.InferRequestedOutput(
    "output", binary_data=True, class_count=4
)

# Querying the server
results = client.infer(model_name="mobilenet_onnx", inputs=[inputs], outputs=[outputs])
inference_output = results.as_numpy("output")
predictions = np.array(inference_output)
processed_results = process_predictions(predictions)
for probability, class_idx, label in processed_results:
    print(f"Label: {label}, Class Index: {class_idx}, Probability: {probability:.6f}")

Label: car, Class Index: 1, Probability: 0.997789
Label: van, Class Index: 3, Probability: 0.002097
Label: truck, Class Index: 2, Probability: 0.000071
Label: bus, Class Index: 0, Probability: 0.000043
